In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune

from scipy import io
import os

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_1.mat')

In [ ]:
K1 = mat_file['K1']
A1 = mat_file['A1']

K2 = mat_file['K2']
A2 = mat_file['A2']

# K1 = np.swapaxes(K1,0,2)
# K1 = np.swapaxes(K1,1,2)
# A1 = np.swapaxes(A1,0,2)
# A1 = np.swapaxes(A1,1,2)

# K2 = np.swapaxes(K2,0,2)
# K2 = np.swapaxes(K2,1,2)
# A2 = np.swapaxes(A2,0,2)
# A2 = np.swapaxes(A2,1,2)


Y1 = mat_file['Y1']
Y2 = mat_file['Y2']

In [ ]:
k1 = torch.FloatTensor(K1)

In [23]:
seq_len = 300
batch_size = 8
hidden_size = 3
lstm_output_size = hidden_size
input_size = 22
n_class = 2

dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, dropout=0.3)
    self.lstm_2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, dropout=0.3)
    self.fc = nn.Linear(hidden_size*2, n_class)

  def forward(self, X, hidden_and_cell):
    outputs1, (h_n1,c_n1) = self.lstm_1(X, hidden_and_cell)
    outputs2, (h_n2,c_n2) = self.lstm_2(X, hidden_and_cell)

    outputs = torch.cat((h_n1[-1],h_n2[-1]), dim=2)  

    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    return model

In [24]:
model = TextLSTM()
print(model)

TextLSTM(
  (lstm_1): LSTM(22, 3, dropout=0.3)
  (lstm_2): LSTM(22, 3, dropout=0.3)
  (fc): Linear(in_features=6, out_features=2, bias=True)
)


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [18]:
input = torch.zeros(n_timeStep, 1, n_channel)
hidden = torch.zeros(1, 1, n_hidden, requires_grad=True)
cell = torch.zeros(1, 1, n_hidden, requires_grad=True)

print("input size: ",input.size())

model(input,(hidden,cell))

input size:  torch.Size([16, 1, 22])


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [20]:
x = torch.randn(1, 6,16)
print(x)

tensor([[[-0.2263,  0.5266, -0.2662, -0.4090,  1.2948,  0.0481, -1.1158,
          -0.1250, -0.9933,  1.0017,  0.5295, -0.0893, -1.3164,  0.6437,
           1.4094,  1.7076],
         [ 0.8200,  0.6360,  0.2685, -0.5715, -1.5548,  0.2716,  0.8671,
          -1.0428, -1.8106,  1.0006,  2.8846, -0.9818, -0.3597,  0.5719,
           0.7379,  1.5514],
         [ 0.0530,  0.0608, -1.3184, -0.1630, -0.1283, -0.1292, -0.7647,
           1.8540,  0.1507,  0.6885,  0.9244,  0.5030,  0.2170, -1.6820,
           0.8214, -1.5395],
         [ 0.8364, -0.0582,  1.1133,  0.2993,  1.6734, -1.9486,  0.5429,
           1.3786,  0.0290,  1.1851, -0.8030,  0.5502, -0.3045, -1.2143,
           0.1386,  0.6613],
         [-0.4581, -1.5599,  0.9319,  1.5936,  0.9931, -0.3526, -0.2520,
           0.0312,  1.1070, -0.6188,  0.1242,  0.6315, -1.2922, -1.9808,
           0.3748, -0.8462],
         [-0.5821, -0.0273, -0.8419, -0.2217,  0.3885,  2.0175, -0.4445,
          -0.3909, -0.4055,  0.5667, -1.1184,  1.578

In [22]:
x[-1].size()

torch.Size([6, 16])